In [597]:
#!pip install streamlit

import sqlite3
import pandas as pd




In [598]:
# Data

dbfile = 'bce.db'
# Create a SQL connection to our SQLite database
con = sqlite3.connect(dbfile)

# creating cursor
cur = con.cursor()

status = """
    SELECT
        Status, (COUNT(*)/18632.92) AS quantity_of_companies
    FROM
        enterprise
    GROUP BY Status
"""

JuridicalForm = """

    SELECT
        JuridicalForm, (COUNT(*)/18632.92) AS quantity_of_companies
    FROM
        enterprise
    --GROUP BY JuridicalForm
    ORDER BY quantity_of_companies DESC

"""
## not working version with names for the juridical forms
JuridicalForm2 = """

    SELECT
        c1.Description, e1.JuridicalForm, COUNT(*) AS quantity_of_companies, COUNT(*) * 100.0 / SUM(COUNT(*)) OVER () AS percentage
    FROM
        enterprise  AS e1 
    LEFT JOIN code as c1 ON  e1.JuridicalForm = c1.Code
	--WHERE e1.JuridicalForm IS NULL
    --AND c1.Language LIKE "FR"
    --AND c1.Category LIKE "JuridicalF%"
    --GROUP BY e1.JuridicalForm
    ORDER BY quantity_of_companies DESC
"""
TypeOfCompany = """

    SELECT
        TypeOfEnterprise, (COUNT(*)/18632.92) AS quantity_of_companies
    FROM
        enterprise
    GROUP BY TypeOfEnterprise
    ORDER BY quantity_of_companies DESC

"""

sector_average_age = """

    SELECT
        c1.Description, a1.NaceCode, AVG(2022 - strftime('%Y',e1.StartDate)) AS "Age", COUNT(EntityNumber) AS Number_of_Activities
    FROM activity AS a1
         
	LEFT JOIN enterprise AS e1 	ON  a1.EntityNumber = e1.EnterpriseNumber
	LEFT JOIN code AS c1 ON a1.NaceCode = c1.Code
	WHERE c1.Language LIKE "FR"
    --AND strftime('%Y',e1.StartDate)>2021
	GROUP BY NaceCode
	ORDER BY Age DESC

"""

status_df = pd.read_sql_query( status   , con)
JuridicalForm_df = pd.read_sql_query( JuridicalForm   , con)
TypeOfCompany_df = pd.read_sql_query( TypeOfCompany   , con)
sector_average_age_df = pd.read_sql_query( sector_average_age   , con)

#sector_average_age_df.to_csv('sector_average_age.csv', index=False)
status_df.to_csv('status.csv', index=False)

status_df


,Status,quantity_of_companies
0,AC,100.0


In [599]:
outer_countries = """
    SELECT CountryFR, COUNT(*) AS Count
    FROM address
    GROUP BY CountryFR
    ORDER BY COUNT(*) DESC
"""
outer_countries_df = pd.read_sql_query( outer_countries   , con)
outer_countries_df.to_csv('outer_countries.csv', index=True)

outer_countries_df


,CountryFR,Count
0,None,2627222
1,Pays-Bas,23053
2,France,12945
3,Allemagne (Rép.féd.),9611
4,Grande-Bretagne,7327
...,...,...
206,Brunei,1
207,Belize(R.U.),1
208,Azerbaïdjan ( Rép. ),1
209,Antigua(R.U.),1


In [600]:
#belgian_countries = """
#    SELECT Zipcode, COUNT(*) AS Count
#    FROM address
#    GROUP BY Zipcode
#    ORDER BY COUNT(*) DESC
#"""
#belgian_countries_df = pd.read_sql_query( belgian_countries   , con)
#
#belgian_countries_df['Zipcode'] = belgian_countries_df['Zipcode'].str.extract('(\d+)', expand=False)
#belgian_countries_df['Zipcode'] = belgian_countries_df['Zipcode'].astype(float)
#
#belgian_countries_df

In [601]:
#belgian_countries_df = belgian_countries_df.dropna()
#belgian_countries_df['Zipcode'] = belgian_countries_df['Zipcode'].astype(int)
#belgian_countries_df['Region'] = belgian_countries_df['Zipcode']
#for zipcode in belgian_countries_df['Region'].unique():
#    if zipcode >= 1000 and zipcode <= 1299:
#        belgian_countries_df['Region'].replace(zipcode, 'Bruxelles', inplace = True)
#    elif zipcode >= 1300 and zipcode <= 1499:
#        belgian_countries_df['Region'].replace(zipcode, 'Wallonia', inplace = True)
#    elif zipcode >= 1500 and zipcode <= 3999:
#        belgian_countries_df['Region'].replace(zipcode, 'Flanders', inplace = True)
#    elif zipcode >= 4000 and zipcode <= 7999:
#        belgian_countries_df['Region'].replace(zipcode, 'Wallonia', inplace = True)
#    elif zipcode >= 8000 and zipcode <= 9999:
#        belgian_countries_df['Region'].replace(zipcode, 'Flanders', inplace = True)
#    else:
#        belgian_countries_df['Region'].replace(zipcode, np.nan, inplace = True)
#
#
##Region	Postal codes	pce
##Bruxelles	1000	1299
##Wallonia 	1300	1499
##Flanders	1500	3999
##Wallonia 	4000	7999
##Flanders	8000	9999
#
#belgian_countries_df = belgian_countries_df.dropna()
#
##belgian_countries_df.to_csv()
#belgian_countries_df = belgian_countries_df.groupby(['Region']).sum()
#belgian_countries_df = belgian_countries_df.Count.sort_values(ascending = False))
#
##belgian_countries_df.to_csv('belgian_regions.csv', index=True)
#belgian_countries_df
#

In [628]:
import numpy as np
JuridicalForm_df = pd.read_csv('JuridicalForm_df_clean.csv')
#
JuridicalForm_df['JuridicalForm']=JuridicalForm_df['JuridicalForm'].replace(np.nan, "Missing")
#JuridicalForm_df['JuridicalForm']=JuridicalForm_df['JuridicalForm'].astype(str)
##JuridicalForm_df.to_csv('JuridicalForm_df_clean2.csv', index=False)
JuridicalForm_df[JuridicalForm_df['quantity_of_companies']<1]

,0,N,JuridicalForm,quantity_of_companies
10,11,10,Société en commandite,0.924600
11,12,11,Société coopérative à responsabilité limitée,0.670158
12,13,12,Entité étrangère sans établissement belge avec...,0.661249
13,14,13,Société coopérative (ancien statut),0.625506
14,15,14,Unité TVA,0.411798
...,...,...,...,...
103,104,103,Services du Premier Ministre,0.000054
104,105,104,Ministère de la Région de Bruxelles-Capitale,0.000054
105,106,105,Société coopérative à responsabilité limitée à...,0.000054
106,107,106,Autre forme à finalité sociale de droit public,0.000054


In [634]:
import plotly.figure_factory as ff
import plotly.express as px
fig = px.bar(JuridicalForm_df, x='JuridicalForm', y='quantity_of_companies', text= 'quantity_of_companies', log_y=True,
                 width=1000, height=800)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=30))
fig.update_yaxes(title_text='Percentage of companies')
fig.update_xaxes(title_text='Juridical Form')
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=1, y1=1, yref="y")

fig.add_annotation( # add a text callout with arrow
    text="Under 1%!", y=0.1, x='Société en commandite', arrowhead=1, showarrow=True)
fig.show()

In [604]:
sector_average_age_df = pd.read_csv( 'sector_average_age.csv')
sector_average_age_df

,Description,NaceCode,Age,Number_of_Activities
0,Enseignement secondaire professionnel ou techn...,80226,93.000000,5
1,Mutuelles et caisses d'assurance soins,75302,76.325000,935
2,Activités des hôpitaux gériatriques,86102,62.200000,25
3,Extraction et agglomération de la houille,10100,60.363636,32
4,Location de logements sociaux,70202,58.356667,2189
...,...,...,...,...
5308,la culture de cônes de houblon et de racines e...,111006,NaN,168
5309,la culture des graines et des fruits oléagineu...,111005,NaN,7878
5310,la culture de tabac (y compris la récolte et l...,111004,NaN,2202
5311,Société en nom collectif,11,NaN,124


In [623]:
#st.markdown('### Sector average age')
fig = px.scatter(sector_average_age_df, x="NaceCode", y="Age", log_x=True, range_x=[1000, 10000000], 
    color = "Number_of_Activities", color_continuous_scale='rainbow' )
fig.update_xaxes(title_text='Nace Code', constrain="domain",)
fig.update_yaxes(title_text='Age')
fig.update_traces(marker=dict(size=10), selector=dict(mode='markers'))
#fig = px.bar(JuridicalForm_df, y='JuridicalForm', x='quantity_of_companies', orientation='h')

fig.show()


In [606]:
# be central type of companies and age Juridical form 

In [607]:
belgian_countries_df = pd.read_csv('belgian_regions.csv')
belgian_countries_df 

,Region,Zipcode,Count
0,Bruxelles,2111966,400744
1,Flanders,36264794,1560072
2,Wallonia,60370634,701667


In [608]:
fig = px.bar(belgian_countries_df, x='Region', y='Count', width=500, height=400, color = "Region" )
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=30))
fig.update_yaxes(title_text='Percentage of companies')
fig.update_xaxes(title_text='Belgian region', categoryorder='array', categoryarray= ['Bruxelles','Wallonia','Flanders'])

fig.show()

In [609]:
outer_countries_df = pd.read_csv('outer_countries.csv')
outer_countries_df

,Unnamed: 0,CountryFR,Count
0,0,NaN,2627222
1,1,Pays-Bas,23053
2,2,France,12945
3,3,Allemagne (Rép.féd.),9611
4,4,Grande-Bretagne,7327
...,...,...,...
206,206,Brunei,1
207,207,Belize(R.U.),1
208,208,Azerbaïdjan ( Rép. ),1
209,209,Antigua(R.U.),1


In [610]:
fig = px.bar(outer_countries_df, x='CountryFR', y='Count', width=1000, height=800, color = "CountryFR" ,log_y=True)
fig.update_layout({'paper_bgcolor':'rgba(0,0,0,0)', 'plot_bgcolor':'rgba(0,0,0,0)'}, showlegend=False, margin=dict(l=20, r=20, t=20, b=30))
fig.update_yaxes(title_text='Percentage of companies')
fig.update_xaxes(title_text='Belgian region', categoryorder='array', categoryarray= ['Bruxelles','Wallonia','Flanders'])

fig.show()